In [24]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [25]:
import statsmodels.formula.api as smf
import os
os.getcwd()
os.chdir('C:/Users/asus1/Desktop/bootcamp/dersler/hafta3')
os.getcwd()
diabetes = pd.read_csv("diabetes.csv")
df=diabetes.copy()
df=df.dropna()
df.head()
df=diabetes.copy()
df=df.dropna()
y=df["Outcome"]
X=df.drop(['Outcome'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.30, random_state = 42)

In [26]:
from sklearn.preprocessing import StandardScaler #data tamamen nümerik değerlerden oluşuyorsa

In [27]:
scaler =StandardScaler()

scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [28]:
X_test_scaled[0:5]

array([[ 0.69748316, -0.70719864, -0.64639893,  0.81207927,  0.95720244,
         0.26575953, -0.11680393,  0.85019217],
       [-0.52953881, -0.27388818,  0.29399563,  0.74746428, -0.6936878 ,
         0.488933  , -0.94192338, -1.03426754],
       [-0.52953881, -0.39769117, -0.31449497, -1.3202154 , -0.6936878 ,
        -0.1543317 , -0.91266382, -1.03426754],
       [ 1.31099414, -0.42864191,  0.57058226, -1.3202154 , -0.6936878 ,
        -0.96825847,  1.12965312,  0.07927683],
       [ 1.00423865,  0.46892976,  1.12375553, -1.3202154 , -0.6936878 ,
        -0.27248236, -0.76051413,  1.44979298]])

In [29]:
from sklearn.neural_network import MLPClassifier
mlpc = MLPClassifier().fit(X_train_scaled, y_train)

C:\Users\asus1\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [30]:
y_pred= mlpc.predict(X_test_scaled)
accuracy_score(y_test,y_pred)

0.7272727272727273

In [31]:
## model tuning

In [32]:
mlpc

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [33]:
?mlpc

In [34]:
mlpc_params = {"alpha": [0.1, 0.01, 0.02, 0.005, 0.0001,0.00001],
              "hidden_layer_sizes": [(10,10,10),
                                     (100,100),
                                     (3,5), 
                                     (5, 3)],
              "solver" : ["lbfgs","adam","sgd"],
              "activation": ["relu","logistic"]}

In [35]:
mlpc= MLPClassifier()
mlpc_cv_model = GridSearchCV(mlpc, mlpc_params, cv=10,n_jobs=-1, verbose=2)
mlpc_cv_model.fit(X_train_scaled, y_train)

Fitting 10 folds for each of 144 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   34.1s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed:  4.2min finished
C:\Users\asus1\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_iter=200,
                                     momentum=0.9, n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_st...
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'activation': ['relu', 'logistic'],
                         '

In [36]:
mlpc_cv_model.best_params_

{'activation': 'logistic',
 'alpha': 0.02,
 'hidden_layer_sizes': (100, 100),
 'solver': 'adam'}

In [49]:
mlpc_tuned = MLPClassifier(activation = "logistic", alpha=0.02, hidden_layer_sizes=(100,100,100), solver = "adam")

In [50]:
mlpc_tuned.fit(X_train_scaled, y_train)
y_pred=mlpc_tuned.predict(X_test_scaled)
accuracy_score(y_test,y_pred)

0.7445887445887446

In [51]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.81      0.79      0.80       151
           1       0.63      0.65      0.64        80

    accuracy                           0.74       231
   macro avg       0.72      0.72      0.72       231
weighted avg       0.75      0.74      0.75       231



SVM (SUPPORT VECTOR MACHINE)

Lineer ilişkiye bakar, lineer bir modeldir, supervised bir modeldir, nerden çizgi atarsam y lermi en iyi ayırırım , hedefi bu 

hyperplanelerde de çalışabilirsin. Kerneller ise kategorikleri ayrıştırmak için gerekli olan modeller aslında. 
Model eğrim y=wx+b. PCA gibi çalıştığını söyleyebiliriz. Kendi içinde ayrışmamışsa kernel modeller kullanılır(line fit edilmeyecek verilerde)

In [56]:
df=diabetes.copy()
df=df.dropna()
df.head()
df=diabetes.copy()
df=df.dropna()
y=df["Outcome"]
X=df.drop(['Outcome'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.30, random_state = 42)

from sklearn.svm import SVC
# rbf circle ile ayrım için kullanılır, defaultu rbf dir.
svc_model =SVC(kernel = "linear").fit(X_train, y_train)

In [58]:
?svc_model

In [59]:
y_pred= svc_model.predict(X_test)

In [60]:
accuracy_score(y_test,y_pred)

0.7445887445887446

In [61]:
#MODEL TUNING
# C'nin defaultu 1 dir.Errorleri optimize etmek için 
svc_params = { "C": np.arange(1,10)}
svc=SVC(kernel="linear")
svc_cv_model = GridSearchCV(svc, svc_params, cv=10, n_jobs=-1, verbose=2)

In [ ]:
svc_cv_model.fit(X_train,y_train)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.9min


In [ ]:
### banka için decisiontree, boostlardan birini, svc, neural network bunlara bak ödevdee